In [25]:
import pandas as pd
import numpy as np
import tensorflow as tf
from trackml.dataset import load_event, load_dataset 
from trackml.randomize import shuffle_hits
from trackml.score import score_event 
import os
import math
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import time
from multiprocessing import Pool
%matplotlib notebook
%run utils.ipynb

In [41]:
hits, cells, particles, truth = load_data_single_event(1050)
hits_from_seeds = np.load('./hits_from_seeds.npy')

In [27]:
def predict_next_hit(hits_so_far):
    n_steps = len(hits_so_far[0])
    n_input_features = 5
    n_neurons = 200
    n_outputs = 3

    X = tf.placeholder(tf.float32, [None, n_steps, n_input_features], name='input')
    
    lstm = tf.contrib.rnn.LSTMCell(num_units = n_neurons, use_peepholes = True)
    lstm_cell = tf.contrib.rnn.OutputProjectionWrapper(lstm, output_size = n_outputs, reuse=tf.AUTO_REUSE)
    rnn_outputs, states = tf.nn.dynamic_rnn(lstm_cell, X, dtype = tf.float32)
    global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')
    print_op = tf.Print(rnn_outputs, [rnn_outputs], name='print_pred')
    init = tf.global_variables_initializer()
    
    with tf.Session() as lstm3sess:

        init.run()
        initial_step = global_step.eval()
        saver = tf.train.Saver()
        writer = tf.summary.FileWriter('./logs/lstm3', lstm3sess.graph)
        ckpt = tf.train.get_checkpoint_state(os.path.dirname('./checkpoints/lstm3/'))
        # if that checkpoint exists, restore from checkpoint
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(lstm3sess, ckpt.model_checkpoint_path)

        x_data = hits_so_far
        test_print = lstm3sess.run(print_op, feed_dict={X:x_data})
        return [test_print[0][-1][0], test_print[0][-1][1], test_print[0][-1][2]]

In [112]:
"""
Encapsulates the NN used for next-hit prediction 
"""
class predict_engine(): 
    N_steps = 10
    N_INPUT_FEATURES = 5
    N_NEURONS = 200
    N_OUTPUTS = 3
    MODEL_PATH = './checkpoints/baseline-idealv4/-33103.meta'
    
    def __init__(self):
        init = tf.global_variables_initializer()
        
        infSess = tf.Session() 
        init.run(session = infSess)
            
        saver = tf.train.import_meta_graph(self.MODEL_PATH)
        graph = saver.restore(infSess,'./checkpoints/baseline-idealv4/-33103' )
        #graph = tf.get_default_graph #hopefully this always points to the right one...
        predict_op = infSess.graph.get_tensor_by_name("print_op:0")
        self.predict_op = predict_op
        self.infSess = infSess 
    
    def pred(self, seed):
        #print(seed)
        pad_zeros = 10 - len(seed)
        x_data = np.array([np.pad(seed, ((0, pad_zeros), (0,0)), 'constant')])
        #print(x_data.shape)
        
        inputs = tf.placeholder(tf.float32, [None, 10, 5])
        input_tensor = self.infSess.graph.get_tensor_by_name('input_ph:0')
        prediction = self.infSess.run(self.predict_op, feed_dict={input_tensor:x_data})
        #print('input was: ')
        #print(x_data)
        #print('\n')
        #print('pred is: ')
        #print(prediction)
        return prediction
    
    def close(self):
        self.infSess.close()

In [121]:
start = time.time()
prediction = predict_engine()
seeds = next_seed(hits_from_seeds)
hits_from_tracks = []
mse_hitid = []
for i in range(5):
    mse_hitid.append([])
    feed_seed = next(seeds)[0]
    for j in range(7):
        print("j: ", j)
        predicted_hit = prediction.pred(feed_seed)[0][j+2]
        predicted_hit_xyz = polar_to_cartesian(predicted_hit[0], predicted_hit[1], predicted_hit[2])
        real_hit_id = closestHit(predicted_hit_xyz[0], predicted_hit_xyz[1], predicted_hit_xyz[2], hits)
        real_hit = get_xyz(real_hit_id)
        l, v, hit_id = get_hit_info(real_hit[0], real_hit[1], real_hit[2]) #change function to take hits df as parameter
        mse = distance_between_two_points(real_hit, polar_to_cartesian(predicted_hit[0], predicted_hit[1], predicted_hit[2]))
        mse_hitid[i].extend([[mse, real_hit_id]])
        test = real_hit.extend([l,v])
        reshaped_real_hit = np.asarray(real_hit).reshape(1, 5)
        feed_seed = np.append(feed_seed, reshaped_real_hit, axis=0)
    hits_from_tracks.append(feed_seed)
end = time.time()
print(end-start)

INFO:tensorflow:Restoring parameters from ./checkpoints/baseline-idealv4/-33103
j:  0
j:  1
j:  2
j:  3
j:  4
j:  5
j:  6
j:  0
j:  1
j:  2
j:  3
j:  4
j:  5
j:  6
j:  0
j:  1
j:  2
j:  3
j:  4
j:  5
j:  6
j:  0
j:  1
j:  2
j:  3
j:  4
j:  5
j:  6
j:  0
j:  1
j:  2
j:  3
j:  4
j:  5
j:  6
88.5917751789093


In [118]:
hits_from_tracks[0][:,0:3]

TypeError: list indices must be integers or slices, not tuple

In [120]:
np.asarray(mse_hitid)

array([[8.36378107e+00, 4.03760000e+04, 3.30705917e+01, 6.94510000e+04,
        5.83602054e+00, 3.49640000e+04, 1.87087485e+01, 2.19870000e+04,
        2.88325814e+00, 2.96810000e+04, 1.47617470e+01, 3.64080000e+04,
        2.69802125e+00, 2.25670000e+04],
       [3.62005544e+01, 7.35600000e+04, 2.92117755e+01, 3.80420000e+04,
        1.74381340e+01, 1.68320000e+04, 1.72168764e+00, 1.67330000e+04,
        1.40368094e+01, 1.74730000e+04, 6.12448631e+00, 1.62900000e+04,
        1.46005738e+01, 1.63980000e+04],
       [2.83386086e+00, 3.85710000e+04, 1.47937821e+01, 6.79530000e+04,
        6.95469451e+00, 2.51980000e+04, 1.83769047e+01, 6.67010000e+04,
        3.60962328e+01, 7.30770000e+04, 6.79235103e+01, 7.92970000e+04,
        1.29529197e+01, 1.64860000e+04],
       [5.17127614e+01, 7.66610000e+04, 1.80422294e+00, 2.28030000e+04,
        7.28764538e+00, 3.59980000e+04, 1.75274868e+01, 4.17900000e+04,
        1.92915541e+01, 7.71490000e+04, 2.12581349e+01, 8.35880000e+04,
        4.503

In [ ]:
pool = Pool(4)
